In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
# %config Completer.use_jedi = False

In [ ]:
from scipy.spatial import KDTree
import numpy as np
from scipy.spatial.distance import cdist

import os
import sys
module_path = "/Users/jmccarty/Data/221205_ipv_workbench/github/IPV_Workbench"
sys.path.insert(0, module_path)
import pandas as pd
import pickle
import json
from itertools import repeat
import multiprocess
from pprint import pprint
import numpy as np
import pvlib
import proplot as pplt
import ipv_workbench as ipv
from ipv_workbench.utilities import utils, circuits, time_utils
from ipv_workbench.devices import devices
from ipv_workbench.simulator import simulations as ipv_sim
from ipv_workbench.simulator import simulations_mp as ipv_sim_mp
from ipv_workbench.simulator import calculations as ipv_calc
from ipv_workbench.visualize import plots as ipv_plots
from ipv_workbench.translators import panelizer, mapping_irradiance
from ipv_workbench.sample import dummies

from tqdm import tqdm
import functools, operator, collections
import multiprocess as mp
import time

In [ ]:
def collect_raw_irradiance(pv_cells_xyz_arr, sensor_pts_xyz_arr, sensor_pts_irradiance_arr):
    # cdist_arr = cdist(pv_cells[0:5,:], sensor_pts_xyz_arr)
    cdist_arr = cdist(pv_cells_xyz_arr, sensor_pts_xyz_arr)
    first = cdist_arr.argsort()[:,0]
    second = cdist_arr.argsort()[:,1]
    third = cdist_arr.argsort()[:,2]
    irrad_cell_mean = (sensor_pts_irradiance_arr[first] + sensor_pts_irradiance_arr[second] + sensor_pts_irradiance_arr[third]) / 3
    return irrad_cell_mean



In [ ]:
# make dummy data
n_sensors = 20000
n_cells = 3000
max_irrad_total = 900
max_loss = 400

sensor_pts_xyz_arr = np.random.rand(n_sensors,3)

# the irradiance values will always have the same length as the first value of the snensor array shape
sensor_pts_irradiance_values_total = np.random.randint(max_irrad_total, size=(1,n_sensors)).flatten()
# make aup a diffuse split
diffuse_component = np.random.randint(max_loss, size=(1,n_sensors)).flatten()
# calculate the direct contribution
sensor_pts_irradiance_values_direct = sensor_pts_irradiance_values_total - diffuse_component

# make some fake cells
pv_cells_xyz_arr = np.random.rand(n_cells,3)


In [ ]:
pv_cells_xyz_arr

array([[0.54009489, 0.56290408, 0.64169585],
       [0.97689171, 0.88989542, 0.31960513],
       [0.76328268, 0.91232465, 0.09492312],
       ...,
       [0.66225613, 0.3003103 , 0.82241669],
       [0.42699981, 0.57425068, 0.55235676],
       [0.87128416, 0.29799136, 0.83975283]])

In [152]:
cell_irradiance_raw_total = collect_raw_irradiance(pv_cells_xyz_arr, sensor_pts_xyz_arr, sensor_pts_irradiance_values_total)
cell_irradiance_raw_direct = collect_raw_irradiance(pv_cells_xyz_arr, sensor_pts_xyz_arr, sensor_pts_irradiance_values_direct)
cell_irradiance_raw_diffuse = cell_irradiance_raw_total - cell_irradiance_raw_direct

In [158]:
normal_vector = (0,-1,0)
hoy = 3999
epw_file = "/Users/jmccarty/Desktop/fluntern.epw"
cover_color = "clear"
time_utils.hoy_to_date(hoy)

Timestamp('2023-06-16 15:00:00')

In [181]:
vfunc=np.vectorize(mapping_irradiance.calculate_effective_irradiance, excluded={2})
cell_irradiance_effective = vfunc(cell_irradiance_raw_direct, cell_irradiance_raw_diffuse, normal_vector, hoy, epw_file, front_cover_color=cover_color)